## Please run this entire notebook sequentially, otherwise some parts might not work.

#### Image conversion algorithm from HEIC to JPG

In [1]:
from PIL import Image
import os
import pyheif

def convert_heic_images_to_jpg(heic_image_paths, destination_folder):
    """
    This is a helper function we wrote in order to convert HEIC images to JPG images.
    It is not used, since its use is currently commented out below.
    It is not needed to generate the final movie pdf since we provided jpg images already.
    """
    converted_files = []

    for heic_image_path in heic_image_paths:
        try:
            # Open the HEIC image using pyheif
            heif_image = pyheif.read(heic_image_path)

            # Generate the destination file path with the new name and JPG extension
            base_filename = os.path.splitext(os.path.basename(heic_image_path))[0]
            jpg_file_path = os.path.join(destination_folder, f"{base_filename}.jpg")

            # Convert and save the image as JPG using Pillow
            image = Image.frombytes(
                heif_image.mode,
                heif_image.size,
                heif_image.data,
                "raw",
                heif_image.mode,
                heif_image.stride,
            )
            image.save(jpg_file_path, "JPEG")

            print(f"Successfully converted and saved {heic_image_path} to {jpg_file_path}")
            converted_files.append(jpg_file_path)

        except Exception as e:
            print(f"An error occurred while processing {heic_image_path}: {str(e)}")

    return converted_files

In [2]:
# Set this path to the root of the movie if conversion is to be performed
movie_path = "/Users/maste/Documents/RevengeOfTheSloth"
JPG_image_loc = os.path.join(movie_path, "Data")

heic_image_paths = [
    os.path.join(movie_path, "RawData", "IMG_0350.HEIC"),
    os.path.join(movie_path, "RawData", "IMG_0396.HEIC"),
    os.path.join(movie_path, "RawData", "IMG_0440.HEIC"),
    os.path.join(movie_path, "RawData", "IMG_0449.HEIC"),
]

# converted_files = convert_heic_images_to_jpg(heic_image_paths, JPG_image_loc)

#### Poster creation

In [3]:
import os
from PIL import Image

def create_posters(background_folder, overlay_images, output_folder, overlay_positions):
    """
    Creates posters by pasting overlay images onto background images and saves them to the output folder.

    :param background_folder: Path to the folder containing background images.
    :param overlay_images: List of overlay image file paths.
    :param output_folder: Output directory where posters will be saved.
    :param overlay_positions: List of overlay positions as (x, y) tuples for each overlay image.
    """
    for background_file in os.listdir(background_folder):
        if background_file.endswith(('.jpg', '.jpeg', '.png')):
            background_path = os.path.join(background_folder, background_file)
            background_image = Image.open(background_path)
            background_width, background_height = background_image.size

            for overlay_image_path, (x, y), resize_factor in zip(overlay_images, overlay_positions, resize_factors):
                overlay_image = Image.open(overlay_image_path)
                overlay_width, overlay_height = overlay_image.size

                # Resize the overlay image by the specified factor
                overlay_width = int(overlay_width * resize_factor)
                overlay_height = int(overlay_height * resize_factor)
                overlay_image = overlay_image.resize((overlay_width, overlay_height), Image.LANCZOS)

                # Calculate the new position for the overlay image
                new_x = background_width - overlay_width -x
                new_y = background_height - overlay_height - y  # Place on the lower edge

                background_image.paste(overlay_image, (new_x, new_y), mask=overlay_image)

            poster_filename = os.path.splitext(background_file)[0] + "_poster.jpg"
            poster_path = os.path.join(output_folder, poster_filename)
            background_image.save(poster_path, "JPEG")

In [4]:
# Define the paths
bg_img_path = os.path.join("Data", "background")
overlay_img_path = os.path.join("Data", "background_removed")
poster_path = os.path.join("Data", "posters")
movie_name_img_path = os.path.join("Data", "movie_name")
bot_img_path = os.path.join("Data", "bot_images")


# List of overlay image paths
overlay_images = [
    os.path.join(overlay_img_path, "IMG_0439.png"),
    os.path.join(overlay_img_path, "IMG_0394.png"),
    os.path.join(movie_name_img_path, "Revenge.jpeg"),
    os.path.join(overlay_img_path, "2.png"),
    os.path.join(overlay_img_path, "3.png"),
    os.path.join(overlay_img_path, "of_the.png"),
    os.path.join(overlay_img_path, "sloth.png"),
    os.path.join(bot_img_path, "4.jpg"),
]

# List of overlay positions as (x, y) tuples and resize factors for each overlay image
overlay_positions = [
    (2450, -500),      # Lower left corner
    (-1000, -650),     # Lower right corner
    (1500, 2600),
    (3900, 2800),
    (900, 2800),
    (2100, 2900),
    (1100, 1900),
    (2100, -20),
]

# List of resize factors for the overlay images
resize_factors = [
    1.1,    # Adjust this factor to resize the first overlay image
    1.1,    # Adjust this factor to resize the second overlay image
    9,
    3,
    3,
    2,
    3.8,
    1.2,
]

# Create the output directory if it doesn't exist
os.makedirs(poster_path, exist_ok=True)

# Create posters
create_posters(bg_img_path, overlay_images, poster_path, overlay_positions)

In [5]:
import cv2
import os

# Load the image
image_path = os.path.join(poster_path, "1_poster.jpg")
image = cv2.imread(image_path)

# Define the first text and position
text1 = "Produced By: Professor Dino"
position1 = (85, 3685)  # (x, y) coordinates for the first text

# Define the second text and position
text2 = "Directed By: Group 7"
position2 = (200, 3850)  # (x, y) coordinates for the second text

font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 6  # Font scale (you can adjust this)
font_color = (255, 255, 255)  # Text color in BGR format (white in this case)
font_thickness = 20  # Thickness of the text

# Add the first text to the image
cv2.putText(image, text1, position1, font, font_scale, font_color, font_thickness)

# Add the second text to the image
cv2.putText(image, text2, position2, font, font_scale, font_color, font_thickness)

# Save the image with both texts
output_image_path = os.path.join(poster_path, "final_poster.jpg")
cv2.imwrite(output_image_path, image)

# Print the path where the image with both texts is saved
print(f"Image with both texts saved as '{output_image_path}'")

Image with both texts saved as 'Data/posters/final_poster.jpg'


#### Image manipulation and transformation

In [6]:
from PIL import Image
import cv2 
from IPython.display import display

In [7]:
def imgcompress_mem(input_img_or_path, k):
    """
    This function resizes either a PIL image, a cv2 image, or an image given a file path.
    It then returns the new image without saving it as a new file.
    """
    if isinstance(input_img_or_path, str):
        img = cv2.imread(input_img_or_path, cv2.IMREAD_UNCHANGED)
    elif isinstance(input_img_or_path, pili.Image):
        img = convert_from_image_to_cv2(input_img_or_path)
    elif isinstance(input_img_or_path, np.ndarray):
        img = input_img_or_path
    else:
        raise ValueError("The input must be either a path to an image, a PIL Image, or a cv2 image.")

    # set the ratio of resized image
    width = int((img.shape[1])/k)
    height = int((img.shape[0])/k)

    # resize the image by resize() function of openCV library
    return cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)

In [8]:
def imgcompress(input_img_or_path, path_out, k):
    """
    This function resizes either a PIL image, a cv2 image, or an image given a file path.
    It then saves the new image as a new file.
    """
    if isinstance(input_img_or_path, str):
        img = cv2.imread(input_img_or_path, cv2.IMREAD_UNCHANGED)
    elif isinstance(input_img_or_path, pili.Image):
        img = convert_from_image_to_cv2(input_img_or_path)
    elif isinstance(input_img_or_path, np.ndarray):
        img = input_img_or_path
    else:
        raise ValueError("The input must be either a path to an image, a PIL Image, or a cv2 image.")
    
    # set the ratio of resized image
    width = int(img.shape[1] / k)
    height = int(img.shape[0] / k)

    # resize the image by resize() function of openCV library
    scaled = cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)

    # save the resized image using imwrite() function of openCV library
    scaled_rgb = cv2.cvtColor(scaled, cv2.COLOR_BGR2RGB)
    cv2.imwrite(path_out, scaled_rgb)

In [9]:
def cartoonize(path_in, k, blur, line, blur_adaptive):
    """
    This function cartoonizes an image, given path.
    The cartoonized image is returned.
    """
    
    imgc = imgcompress_mem(path_in, k)

    blur_value = blur
    line_size = line
    gray = cv2.cvtColor(imgc, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.medianBlur(gray, blur_value)
    bigedges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_size, blur_adaptive)
    bigedges_pil = cv2.cvtColor(bigedges, cv2.COLOR_BGR2RGB) # Converting BGR to RGB

    return cv2.bitwise_and(imgc, imgc, mask=bigedges)

In [10]:
import numpy as np
import pandas as pd
from PIL import Image as pili, ImageDraw as pild, ImageFont as pilf, ImageOps as piliops

TINT_COLOR = (0, 0, 0)
OPACITY = int(255 * .50)
FONT = pilf.truetype("Fonts/DejaVuSans.ttf", 4)
IMG_BASE_WIDTH = 600
IMG_NUMBERS = 47

In [11]:
def convert_from_cv2_to_image(img: np.ndarray) -> pili:
    """
    This is a simple conversion function between a cv2 image to PIL image
    """
    return Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [12]:
def convert_from_image_to_cv2(img: pili) -> np.ndarray:
    """
    This is a simple conversion function between a PIL image to cv2 image
    """
    return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

In [77]:
import textwrap

def add_text(input_img_or_path, k, text, nlines, font):
    """
    This function adds text to the image.
    It can be used with both a PIL Image as well as file path.
    """
    
    if isinstance(input_img_or_path, pili.Image):
        img = convert_from_image_to_cv2(input_img_or_path)
        
        x_rect_offset = 0
        y_rect_top_offset = -5
        y_rect_bot_offset = 30
        x_text_offset = 100
        y_text_offset = -120
        
    elif isinstance(input_img_or_path, str):
        img = cv2.imread(input_img_or_path)
        
        x_rect_offset = 0
        y_rect_top_offset = -5
        y_rect_bot_offset = 30
        x_text_offset = 0
        y_text_offset = -30
        
    elif isinstance(input_img_or_path, np.ndarray):
        img = input_img_or_path
        
        x_rect_offset = 0
        y_rect_top_offset = -5
        y_rect_bot_offset = 30
        x_text_offset = 0
        y_text_offset = -30
        
    else:
        raise ValueError("The input must be either a PIL Image, a path to an image, or a cv2 image.")
    
    if 0 == len(text):
        return img
    
    print('Using font ' + font + '...')
    myfont = pilf.truetype("Fonts/" + font + ".ttf", 150)

    cblimg_pil = pili.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGBA))

    overlay = pili.new('RGBA', cblimg_pil.size, TINT_COLOR+(0,))
    draw = pild.Draw(overlay)
    
    bbox = myfont.getbbox(text)
    w, h = bbox[2] - bbox[0], bbox[3] - bbox[1]
    
    """
    This is important since it automatically calculates the number of lines, given maximum character length.
    The character length is currently hardcoded, but this can be easily changed.
    """
    wrapped_text = textwrap.fill(text, 45)
    num_lines = wrapped_text.count('\n') + 1
    text = wrapped_text
    
    x, y = 0 + x_text_offset, cblimg_pil.height - (num_lines) * h + y_text_offset
    draw.rectangle((x + x_rect_offset, y + y_rect_top_offset, x + cblimg_pil.width, y + (num_lines) * h + y_rect_bot_offset), fill=TINT_COLOR+(OPACITY,))
    
    draw.text((x+10, y), text, fill=(248,248,248), font=myfont)

    cblimg_pil = pili.alpha_composite(cblimg_pil, overlay)
    cblimg_pil = cblimg_pil.convert("RGB")

    return convert_from_image_to_cv2(cblimg_pil) 

In [62]:
from PIL import Image as pili, ImageOps as piliops

def add_border(input_img_or_path):
    """
    This function adds border to the image, returning the new image.
    """
    
    if isinstance(input_img_or_path, pili.Image):
        img = input_img_or_path
    elif isinstance(input_img_or_path, str):
        img = pili.open(input_img_or_path)
    elif isinstance(input_img_or_path, np.ndarray):
        img = convert_from_cv2_to_image(input_img_or_path)
    else:
        raise ValueError("The input must be either a PIL Image, a path to an image, or a cv2 image.")
    
    return piliops.expand(img, border=(100,100), fill='black')

In [63]:
def get_number_of_lines(text):
    return text.count('\n') + 1 if text else 0

#### The following cell is the proper image transformation sequence

In [64]:
def transform_image(path_in, path_out, text, font = 'DejaVuSans'):
    """
    This function applies a number of transformations, given text and font, to an image.
    It then saves the resulting image as a new file.
    """
    
    cartoonized_image = cartoonize(path_in, 1, 19, 53, 13)
    
    bordered_image = add_border(cartoonized_image)
    
    num_lines = get_number_of_lines(text)
    cbltimg = add_text(bordered_image, 1, text, num_lines, font)
    
    cbltimg_pil = cv2.cvtColor(cbltimg, cv2.COLOR_BGR2RGB)
    imgcompress(cbltimg_pil, path_out, 1)

#### Stitching

In [65]:
import numpy as np
import PIL

def horizontal_stitch(list_im_paths, path_out):
    """
    This function horizontally stitches images in a given list of file paths.
    It then saves the resulting strip.
    """
    imgs = [ PIL.Image.open(i) for i in list_im_paths ]

    # pick the image which is the smallest, and resize the others to match it (can be arbitrary image shape here)
    min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
    imgs_comb = np.hstack( [np.asarray( i.resize(min_shape) ) for i in imgs] )

    # save that beautiful picture
    imgs_comb = PIL.Image.fromarray(imgs_comb)
    imgs_comb.save(path_out)

In [66]:
def vertical_stitch(list_im_paths, path_out):
    """
    This function vertically stitches images in a given list of file paths.
    It then saves the resulting strip.
    """
    imgs = [ PIL.Image.open(i) for i in list_im_paths ]
    
    # pick the image which is the smallest, and resize the others to match it (can be arbitrary image shape here)
    min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
    imgs_comb = np.vstack( [np.asarray( i.resize(min_shape) ) for i in imgs] )
    
    # save that beautiful picture
    imgs_comb = PIL.Image.fromarray(imgs_comb)
    imgs_comb.save(path_out) 

#### PDFing

In [67]:
from PIL import Image
from fpdf import FPDF
from PyPDF2 import PdfFileMerger, PdfFileReader
import os

def make_pdf_hindi():
    # Poster page
    poster_img = 'Data/posters/final_poster.jpg'
    
    width, height = 612, 792
    poster_pdf = FPDF(unit="pt", format=[width, height])
    
    poster_pdf.add_page(orientation='L')
    
    poster_pdf.image(poster_img, 0, 0, height, width)
    poster_pdf.output('Pdfs/PosterTemp.pdf', "F")
    
    # PDF with 6 images
    directory_path = "ImagesHindi/Combined/Vertical/"
    image_paths = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.jpg')]
    image_paths.sort(key=lambda f: int(os.path.splitext(os.path.basename(f))[0]))
    
    images_pdf = FPDF(unit="pt", format=[612, 792])
    
    for img_path in image_paths:
        images_pdf.add_page()
        images_pdf.image(img_path, 0, 0, 612, 792)
    
    images_pdf.output('Pdfs/ImagesHindiTemp.pdf', "F")
    
    # Merge the PDFs
    merger = PdfFileMerger()
    merger.append(PdfFileReader('Pdfs/PosterTemp.pdf', 'rb'))
    merger.append(PdfFileReader('Pdfs/ImagesHindiTemp.pdf', 'rb'))
    merger.write('Pdfs/RevengeOfTheSlothHindi.pdf')
    merger.close()

    # Remove temp pdfs
    os.remove('Pdfs/PosterTemp.pdf')
    os.remove('Pdfs/ImagesHindiTemp.pdf')
    
    print("Hindi done!")

In [68]:
from PIL import Image
from fpdf import FPDF
from PyPDF2 import PdfFileMerger, PdfFileReader
import os

def make_pdf():
    """
    This function is responsible for creating final PDF of the movie out of the provided images.
    The intro page part of the function is commented out to save resources, but can always be enabled.
    The function requires PyPDF2 library as well as fpdf in order to function.
    This is to ensure that the poster page looks good by creating it as a separate landscape pdf
    and then merging the two pdfs together.
    """
    
    # Poster page
    poster_img = 'Data/posters/final_poster.jpg'
    
    width, height = 612, 792
    poster_pdf = FPDF(unit="pt", format=[width, height])
    
    poster_pdf.add_page(orientation='L')
    
    poster_pdf.image(poster_img, 0, 0, height, width)
    poster_pdf.output('Pdfs/PosterTemp.pdf', "F")
    
    ## Intro page
    #width, height = 612, 792
    #intro_img = 'Images/Intro/1.jpg'
    #intro_text = 'Text/intro_text.txt'
    #
    #intro_pdf = FPDF(unit="pt", format=[width, height])
    #intro_pdf.add_page(orientation='P')
#
    ## Open image and get its size
    #img = Image.open(intro_img)
    #img_width, img_height = img.size
    #    
    ## Scale values to fit the image on the page width while maintaining aspect ratio
    #aspect_ratio = img_width / img_height
    #new_width = width
    #new_height = width / aspect_ratio
#
    ## If the calculated height exceeds the page height, scale the image down further
    #if new_height > height:
    #    new_height = height
    #    new_width = height * aspect_ratio
#
    #x_offset = (width - new_width) * 0.5
    #y_offset = 0  # Setting it to zero ensures the image is at the top
#
    #intro_pdf.image(intro_img, x_offset, y_offset, new_width, new_height)
    #
    #with open(intro_text, 'r') as txt_file:
    #        text_content = txt_file.read()
#
    ## Adding text below the image
    #text_y_position = new_height + 40  # 30 is a margin in points; you can adjust as needed
    #intro_pdf.set_font('Arial', size=12)  # You can choose another font and size if you prefer
    #intro_pdf.set_xy(50, text_y_position)  # 50 is the x-coordinate (margin) for the text
    #intro_pdf.multi_cell(0, 10, text_content)  # 10 is the height of each line; adjust as needed
#
    #intro_pdf.output('Pdfs/IntroTemp.pdf', "F")
    
    # PDF with 6 images
    directory_path = "Images/Combined/Vertical/"
    image_paths = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.jpg')]
    image_paths.sort(key=lambda f: int(os.path.splitext(os.path.basename(f))[0]))
    
    images_pdf = FPDF(unit="pt", format=[612, 792])
    
    for img_path in image_paths:
        images_pdf.add_page()
        images_pdf.image(img_path, 0, 0, 612, 792)
    
    images_pdf.output('Pdfs/ImagesTemp.pdf', "F")
    
    # Merge the PDFs
    merger = PdfFileMerger()
    merger.append(PdfFileReader('Pdfs/PosterTemp.pdf', 'rb'))
    #merger.append(PdfFileReader('Pdfs/IntroTemp.pdf', 'rb'))
    merger.append(PdfFileReader('Pdfs/ImagesTemp.pdf', 'rb'))
    merger.write('Pdfs/RevengeOfTheSloth.pdf')
    merger.close()

    # Remove temp pdfs
    os.remove('Pdfs/PosterTemp.pdf')
    #os.remove('Pdfs/IntroTemp.pdf')
    os.remove('Pdfs/ImagesTemp.pdf')
    
    print("done!")

#### Random story book

In [69]:
def create_random_list(number_of_images = 100):
    """
    This function creates a list of lists. The list elements are indices of a different list.
    The entire goal is to create lists of random length from 3 to 5 elements. Up to number_of_images.
    This structure is then used to create random-length horizontal image strips.
    """

    list_im = [x for x in range(number_of_images+1)]
    rnd_stitch_list = []
   
    while 0 < len(list_im):
        lines = 0
        num_cols = old_num_cols = 0
        while lines < 6 and 0 < len(list_im):
            lines += 1
            while num_cols == old_num_cols:
                num_cols = np.random.randint(3, 5)
            old_num_cols = num_cols
            window = list_im[:num_cols]
            rnd_stitch_list.append(window)
            list_im = list_im[num_cols:]
    
    return rnd_stitch_list

#### These are image transformation functions, that take all of the images and transform them, add text, and stitch them together

In [81]:
import pandas as pd
import numpy as np

def transfrom_images_hindi():
    """
    This function is similar to the one below. It creates Hindi-translated movie images.
    """
    # Movie pages
    path_in_root = 'Data/'
    path_out_root = 'ImagesHindi/Combined/'
    
    text_data = pd.read_csv('Text/Text_Data.csv')
    list_im_paths = []
    
    """
    This goes through all of the images according to paths given in 'IMG_ID' column in the .csv text data file
    It essentially simply creates a path to each image, applies transformations such as border, filters, and text
    and then it saves the image and the path to it. 
    """
    for index, row in text_data.iterrows():
        processed_text = row['HIN']#.replace('\\n', '\n')
        transform_image(path_in_root + row['IMG_ID'], path_out_root + row['IMG_ID'], processed_text, 'NirmalaB')
        list_im_paths.append(path_out_root + row['IMG_ID'])
    
    """
    This goes through all of the transformed images and horizontally stitches them in a random fashion.
    Each stitched set of images is saved along with its path.
    """
    rnd_stitch_list = create_random_list(len(list_im_paths)-1)
    vertical_paths = []
    for i, horizontal_list in enumerate(rnd_stitch_list):
        print('Horizontal stitch')
        horizontal_list_paths = [list_im_paths[idx] for idx in horizontal_list]
        horizontal_stitch(horizontal_list_paths, path_out_root + str(i) + ".jpg")
        vertical_paths.append(path_out_root + str(i) + ".jpg")
    
    """
    This vertically stitches sets of 6 horizontally-stitched images together, forming one image.
    Each vertically-stiched image corresponds to one pdf page.
    """
    final_paths = []
    for i in range(0, len(vertical_paths), 6):
        print('Vertical stitch')
        sublist = vertical_paths[i:i+6]
        vertical_stitch(sublist, path_out_root + "Vertical/" + str(i) + ".jpg")
        final_paths.append(path_out_root + "Vertical/" + str(i) + ".jpg")

In [82]:
import pandas as pd
import numpy as np

def transfrom_images():
    # Intro page
    path_in_root = 'Data/'
    path_out_root = 'Images/Intro/'
    
    text_data = pd.read_csv('Text/Intro_Data.csv')
    list_im_paths = []
    
    """
    This goes through all of the images according to paths given in 'IMG_ID' column in the .csv intro data file
    It essentially simply creates a path to each image, applies transformations such as border, filters, and text
    and then it saves the image and the path to it.
    """
    for index, row in text_data.iterrows():
        processed_text = row['ENG']#.replace('\\n', '\n')
        transform_image(path_in_root + row['IMG_ID'], path_out_root + row['IMG_ID'], processed_text)
        list_im_paths.append(path_out_root + row['IMG_ID'])
        
    horizontal_stitch(list_im_paths, path_out_root + "1.jpg")
    
    # Movie pages
    path_in_root = 'Data/'
    path_out_root = 'Images/Combined/'
    
    text_data = pd.read_csv('Text/Text_Data.csv')
    list_im_paths = []
    
    """
    This goes through all of the images according to paths given in 'IMG_ID' column in the .csv text data file
    It essentially simply creates a path to each image, applies transformations such as border, filters, and text
    and then it saves the image and the path to it.
    """
    for index, row in text_data.iterrows():
        processed_text = row['ENG']#.replace('\\n', '\n')
        transform_image(path_in_root + row['IMG_ID'], path_out_root + row['IMG_ID'], processed_text)
        list_im_paths.append(path_out_root + row['IMG_ID'])
    
    """
    This goes through all of the transformed images and horizontally stitches them in a random fashion.
    Each stitched set of images is saved along with its path.
    """
    rnd_stitch_list = create_random_list(len(list_im_paths)-1)
    vertical_paths = []
    for i, horizontal_list in enumerate(rnd_stitch_list):
        print('Horizontal stitch')
        horizontal_list_paths = [list_im_paths[idx] for idx in horizontal_list]
        horizontal_stitch(horizontal_list_paths, path_out_root + str(i) + ".jpg")
        vertical_paths.append(path_out_root + str(i) + ".jpg")
    
    """
    This vertically stitches sets of 6 horizontally-stitched images together, forming one image.
    Each vertically-stiched image corresponds to one pdf page.
    """
    final_paths = []
    for i in range(0, len(vertical_paths), 6):
        print('Vertical stitch')
        sublist = vertical_paths[i:i+6]
        vertical_stitch(sublist, path_out_root + "Vertical/" + str(i) + ".jpg")
        final_paths.append(path_out_root + "Vertical/" + str(i) + ".jpg")
        

#### This is the main entry-point
You can uncomment the two Hindi lines to generate a Hindi-translated movie PDF. Commented out by default to save space and time since PDF generation takes a lot of resources.

In [83]:
transfrom_images()
make_pdf()
#transfrom_images_hindi()
#make_pdf_hindi()

Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
Using font DejaVuSans...
